In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from transformers import ViTModel, ViTConfig
import numpy as np
from sklearn.preprocessing import StandardScaler
from torch_geometric.data import Data
from codecarbon import track_emissions
import psutil
import time
import pandas as pd
from contextlib import AbstractContextManager

# Custom context manager class for resource monitoring
class ResourceMonitor(AbstractContextManager):
    def __init__(self, module_name):
        self.module_name = module_name
        self.metrics = None

    def __enter__(self):
        self.start_time = time.time()
        self.start_cpu = psutil.cpu_percent(interval=0.1)  # Small interval for better accuracy
        self.start_mem = psutil.Process().memory_info().rss / 1024**2  # MB
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        time.sleep(0.01)  # Small delay to ensure resource usage is captured
        end_time = time.time()
        end_cpu = psutil.cpu_percent(interval=0.1)
        end_mem = psutil.Process().memory_info().rss / 1024**2  # MB
        self.metrics = {
            'module': self.module_name,
            'cpu_usage_percent': max((end_cpu + self.start_cpu) / 2, 0.0),  # Avoid negative values
            'memory_usage_mb': max(end_mem - self.start_mem, 0.0)  # Avoid negative values
        }
        print(f"Debug: {self.module_name} metrics - CPU: {self.metrics['cpu_usage_percent']:.2f}%, RAM: {self.metrics['memory_usage_mb']:.2f} MB")
        return False  # Allow exceptions to propagate

    def get_metrics(self):
        if self.metrics is None:
            print(f"Warning: No metrics collected for {self.module_name}")
            return {'module': self.module_name, 'cpu_usage_percent': 0.0, 'memory_usage_mb': 0.0}
        return self.metrics

# Preprocess sMRI
def preprocess_smri_slices(smri_data):
    scaler = StandardScaler()
    N, V, C, H, W = smri_data.shape
    processed_slices = []
    for view in range(V):  # Axial, coronal, sagittal
        slices = smri_data[:, view, 0, :, :]
        slices = scaler.fit_transform(slices.reshape(N, -1)).reshape(N, H, W)
        processed_slices.append(torch.tensor(slices, dtype=torch.float32).unsqueeze(1))
    return torch.stack(processed_slices, dim=1)

# Preprocess phenotype data
def preprocess_phenotype_data(phenotype_data, feature_names):
    scaler = StandardScaler()
    phenotype_data = scaler.fit_transform(phenotype_data)
    graph_data_list = []
    for sample in phenotype_data:
        x = torch.tensor(sample, dtype=torch.float32).reshape(1, -1)
        edge_index = torch.tensor([[0], [0]], dtype=torch.long)
        graph_data = Data(x=x, edge_index=edge_index)
        graph_data_list.append(graph_data)
    return graph_data_list

# Multi-view ViT
class MultiViewViT(nn.Module):
    def __init__(self, image_size=128, patch_size=16, num_classes=768):
        super(MultiViewViT, self).__init__()
        vit_config = ViTConfig(
            image_size=image_size,
            patch_size=patch_size,
            hidden_size=256,
            num_attention_heads=8,
            num_channels=1
        )
        self.vits = nn.ModuleList([ViTModel(vit_config) for _ in range(3)])
        self.attention = nn.MultiheadAttention(embed_dim=256*3, num_heads=8)
        self.fc = nn.Linear(256*3, num_classes)

    def forward(self, x):
        if x.shape[2] != 1:
            raise ValueError(f"Expected 1 channel, got {x.shape[2]}")
        view_features = []
        for i, vit in enumerate(self.vits):
            view_x = x[:, i, :, :, :]
            vit_out = vit(view_x).last_hidden_state[:, 0, :]
            view_features.append(vit_out)
        view_features = torch.cat(view_features, dim=-1)
        view_features = view_features.unsqueeze(0)
        attn_output, _ = self.attention(view_features, view_features, view_features)
        attn_output = attn_output.squeeze(0)
        return self.fc(attn_output)

# GNN
class PhenotypeGNN(nn.Module):
    def __init__(self, num_features, hidden_dim=128, output_dim=256):
        super(PhenotypeGNN, self).__init__()
        self.conv1 = GCNConv(num_features, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)
        self.fc = nn.Linear(output_dim, output_dim)

    def forward(self, data_list):
        batch_features = []
        for data in data_list:
            x, edge_index = data.x, data.edge_index
            x = F.relu(self.conv1(x, edge_index))
            x = F.relu(self.conv2(x, edge_index))
            x = self.fc(x.squeeze(0))
            batch_features.append(x)
        return torch.stack(batch_features, dim=0)

# Cross-Modal Transformer
class CrossModalTransformer(nn.Module):
    def __init__(self, embed_dim=768, num_heads=8):
        super(CrossModalTransformer, self).__init__()
        self.cross_attention = nn.MultiheadAttention(embed_dim=embed_dim, num_heads=num_heads)
        self.norm = nn.LayerNorm(embed_dim)
        self.fc_phenotype = nn.Linear(256, embed_dim)

    def forward(self, smri_features, phenotype_features):
        phenotype_features = self.fc_phenotype(phenotype_features)
        combined = torch.stack([smri_features, phenotype_features], dim=0)
        attn_output, _ = self.cross_attention(combined, combined, combined)
        fused = self.norm(attn_output[0] + attn_output[1])
        return fused

# Ensemble with Uncertainty
class NeuralEnsemble(nn.Module):
    def __init__(self, input_dim=768, hidden_dim=128, num_classes=2, num_estimators=5):
        super(NeuralEnsemble, self).__init__()
        self.estimators = nn.ModuleList([
            nn.Sequential(
                nn.Linear(input_dim, hidden_dim),
                nn.ReLU(),
                nn.Dropout(0.3),
                nn.Linear(hidden_dim, num_classes)
            ) for _ in range(num_estimators)
        ])

    def forward(self, x, training=True):
        outputs = []
        for estimator in self.estimators:
            estimator.train(training)
            outputs.append(estimator(x))
        outputs = torch.stack(outputs, dim=0)
        mean_output = torch.mean(outputs, dim=0)
        uncertainty = torch.std(outputs, dim=0).mean(dim=-1)
        return mean_output, uncertainty

# Complete Model
class TransGraphASD(nn.Module):
    def __init__(self, image_size=128, patch_size=16, phenotype_features=10):
        super(TransGraphASD, self).__init__()
        self.mv_vit = MultiViewViT(image_size, patch_size)
        self.gnn = PhenotypeGNN(num_features=phenotype_features)
        self.cmt = CrossModalTransformer()
        self.ensemble = NeuralEnsemble()

    def forward(self, smri_data, phenotype_data, training=True):
        batch_resources = []
        
        with ResourceMonitor('MultiViewViT') as smri_ctx:
            smri_features = self.mv_vit(smri_data)
            batch_resources.append(smri_ctx.get_metrics())
        
        with ResourceMonitor('PhenotypeGNN') as gnn_ctx:
            phenotype_features = self.gnn(phenotype_data)
            batch_resources.append(gnn_ctx.get_metrics())
        
        with ResourceMonitor('CrossModalTransformer') as cmt_ctx:
            fused_features = self.cmt(smri_features, phenotype_features)
            batch_resources.append(cmt_ctx.get_metrics())
        
        with ResourceMonitor('NeuralEnsemble') as ensemble_ctx:
            output, uncertainty = self.ensemble(fused_features, training)
            batch_resources.append(ensemble_ctx.get_metrics())
        
        return output, uncertainty, batch_resources

# Contrastive Loss
def contrastive_loss_fn(smri_features, phenotype_features):
    assert smri_features.shape[1] == phenotype_features.shape[1]
    similarity = F.cosine_similarity(smri_features, phenotype_features, dim=-1)
    return -torch.mean(similarity)

# Function to print resource utilization summary
def print_resource_summary(resource_logs, phase="Training"):
    if not resource_logs:
        print(f"\n{phase} Resource Utilization Summary: No data available")
        return
    print(f"\n{phase} Resource Utilization Summary:")
    print("-" * 40)
    resource_df = pd.DataFrame([r for r in resource_logs if r and 'module' in r])
    if resource_df.empty or 'module' not in resource_df.columns:
        print("Error: No valid resource logs with 'module' key")
        return
    for module in resource_df['module'].unique():
        module_data = resource_df[resource_df['module'] == module]
        avg_cpu = module_data['cpu_usage_percent'].mean()
        avg_mem = module_data['memory_usage_mb'].mean()
        print(f"{module}:")
        print(f"  Average CPU Usage: {avg_cpu:.2f}%")
        print(f"  Average RAM Usage: {avg_mem:.2f} MB")
    print("-" * 40)

# Training Function with Resource Monitoring
@track_emissions(project_name="TransGraph-ASD Training", country_iso_code="IND")
def train_model(model, smri_data, phenotype_data, labels, num_epochs=1, batch_size=64):
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    criterion = nn.CrossEntropyLoss()
    resource_logs = []

    for epoch in range(num_epochs):
        model.train()
        epoch_resources = []
        total_loss = 0.0

        for i in range(0, len(smri_data), batch_size):
            batch_smri = smri_data[i:i+batch_size]
            batch_phenotype = phenotype_data[i:i+batch_size]
            batch_labels = labels[i:i+batch_size]

            optimizer.zero_grad()
            outputs, _, batch_resources = model(batch_smri, batch_phenotype)
            loss = criterion(outputs, batch_labels)

            smri_features = model.mv_vit(batch_smri)
            phenotype_features = model.gnn(batch_phenotype)
            with torch.no_grad():
                aligned_phenotype_features = model.cmt.fc_phenotype(phenotype_features)
            contrastive_loss = contrastive_loss_fn(smri_features, aligned_phenotype_features)
            total_loss = loss + 0.1 * contrastive_loss

            total_loss.backward()
            optimizer.step()

            for resource in batch_resources:
                if resource and 'module' in resource:
                    resource['epoch'] = epoch + 1
                    resource['batch'] = i // batch_size + 1
                    epoch_resources.append(resource)
                else:
                    print(f"Warning: Invalid resource entry in batch {i // batch_size + 1}")

        avg_loss = total_loss.item() / (len(smri_data) // batch_size)
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}")
        
        print_resource_summary(epoch_resources, phase=f"Training Epoch {epoch+1}")
        resource_logs.extend(epoch_resources)

    pd.DataFrame(resource_logs).to_csv('resource_usage.csv', index=False)
    return resource_logs

# Inference with Uncertainty
def predict_with_uncertainty(model, smri_data, phenotype_data, num_mc_samples=10):
    model.eval()
    predictions = []
    uncertainties = []
    resource_logs = []

    for i in range(num_mc_samples):
        with torch.no_grad():
            output, uncertainty, batch_resources = model(smri_data, phenotype_data, training=True)
            predictions.append(F.softmax(output, dim=-1))
            uncertainties.append(uncertainty)
            for resource in batch_resources:
                if resource and 'module' in resource:
                    resource['sample'] = i + 1
                    resource_logs.append(resource)
                else:
                    print(f"Warning: Invalid resource entry in MC sample {i + 1}")

    predictions = torch.mean(torch.stack(predictions), dim=0)
    uncertainties = torch.mean(torch.stack(uncertainties), dim=0)

    print_resource_summary(resource_logs, phase="Inference")
    return predictions, uncertainties, resource_logs

# Main Execution
if __name__ == "__main__":
    N, H, W, num_features = 100, 128, 128, 20  # Increased data size for measurable resource usage
    smri_data = torch.randn(N, 3, 1, H, W)
    phenotype_data = preprocess_phenotype_data(np.random.randn(N, num_features), [f"f{i}" for i in range(num_features)])
    labels = torch.randint(0, 2, (N,))

    model = TransGraphASD(image_size=H, patch_size=16, phenotype_features=num_features)
    train_resource_logs = train_model(model, smri_data, phenotype_data, labels, num_epochs=1, batch_size=64)
    predictions, uncertainties, inference_resource_logs = predict_with_uncertainty(model, smri_data, phenotype_data)
    print("Predictions:", predictions)
    print("Uncertainties:", uncertainties)

[codecarbon WARNING @ 14:45:27] Multiple instances of codecarbon are allowed to run at the same time.
[codecarbon INFO @ 14:45:27] [setup] RAM Tracking...
[codecarbon INFO @ 14:45:27] [setup] CPU Tracking...
[codecarbon WARNING @ 14:45:30] No CPU tracking mode found. Falling back on estimation based on TDP for CPU. 
 Windows OS detected: Please install Intel Power Gadget to measure CPU

[codecarbon INFO @ 14:45:30] CPU Model on constant consumption mode: 11th Gen Intel(R) Core(TM) i5-1135G7 @ 2.40GHz
[codecarbon WARNING @ 14:45:30] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 14:45:30] [setup] GPU Tracking...
[codecarbon INFO @ 14:45:30] No GPU found.
[codecarbon INFO @ 14:45:30] The below tracking methods have been set up:
                RAM Tracking Method: RAM power estimation model
                CPU Tracking Method: global constant
                GPU Tracking Method: Unspecified
            
[codecarbon INFO @ 14:45:30] >>> Tracker's metadat

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from transformers import ViTModel, ViTConfig
import numpy as np
from sklearn.preprocessing import StandardScaler
from torch_geometric.data import Data
import psutil
import time
import pandas as pd
from contextlib import AbstractContextManager
from codecarbon import EmissionsTracker

# Resource Monitor
class ResourceMonitor(AbstractContextManager):
    def __init__(self, module_name):
        self.module_name = module_name
        self.metrics = None

    def __enter__(self):
        self.start_time = time.time()
        self.start_cpu = psutil.cpu_percent(interval=0.1)
        self.start_mem = psutil.Process().memory_info().rss / 1024**2
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        time.sleep(0.01)
        end_time = time.time()
        end_cpu = psutil.cpu_percent(interval=0.1)
        end_mem = psutil.Process().memory_info().rss / 1024**2
        self.metrics = {
            'module': self.module_name,
            'cpu_usage_percent': max((end_cpu + self.start_cpu) / 2, 0.0),
            'memory_usage_mb': max(end_mem - self.start_mem, 0.0)
        }
        print(f"[DEBUG] {self.module_name}: start_cpu={self.start_cpu}, end_cpu={end_cpu}, start_mem={self.start_mem:.2f}, end_mem={end_mem:.2f}, duration={end_time - self.start_time:.2f}s")
        return False

    def get_metrics(self):
        return self.metrics or {'module': self.module_name, 'cpu_usage_percent': 0.0, 'memory_usage_mb': 0.0}

# Data Preprocessing

def preprocess_smri_slices(smri_data):
    scaler = StandardScaler()
    N, V, C, H, W = smri_data.shape
    processed = []
    for view in range(V):
        slices = smri_data[:, view, 0, :, :]
        slices = scaler.fit_transform(slices.reshape(N, -1)).reshape(N, H, W)
        processed.append(torch.tensor(slices, dtype=torch.float32).unsqueeze(1))
    return torch.stack(processed, dim=1)

def preprocess_phenotype_data(phenotype_data, feature_names):
    scaler = StandardScaler()
    phenotype_data = scaler.fit_transform(phenotype_data)
    graph_data_list = []
    for sample in phenotype_data:
        x = torch.tensor(sample, dtype=torch.float32).reshape(1, -1)
        edge_index = torch.tensor([[0], [0]], dtype=torch.long)
        graph_data = Data(x=x, edge_index=edge_index)
        graph_data_list.append(graph_data)
    return graph_data_list

# Models
class MultiViewViT(nn.Module):
    def __init__(self, image_size=128, patch_size=16, num_classes=768):
        super().__init__()
        vit_config = ViTConfig(
            image_size=image_size, patch_size=patch_size,
            hidden_size=256, num_attention_heads=8, num_channels=1
        )
        self.vits = nn.ModuleList([ViTModel(vit_config) for _ in range(3)])
        self.attn = nn.MultiheadAttention(embed_dim=256 * 3, num_heads=8)
        self.fc = nn.Linear(256 * 3, num_classes)

    def forward(self, x):
        view_features = []
        for i, vit in enumerate(self.vits):
            view_x = x[:, i, :, :, :]
            vit_out = vit(view_x).last_hidden_state[:, 0, :]
            view_features.append(vit_out)
        combined = torch.cat(view_features, dim=-1).unsqueeze(0)
        attn_out, _ = self.attn(combined, combined, combined)
        return self.fc(attn_out.squeeze(0))

class PhenotypeGNN(nn.Module):
    def __init__(self, num_features, hidden_dim=128, output_dim=256):
        super().__init__()
        self.conv1 = GCNConv(num_features, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)
        self.fc = nn.Linear(output_dim, output_dim)

    def forward(self, data_list):
        batch = []
        for data in data_list:
            x = F.relu(self.conv1(data.x, data.edge_index))
            x = F.relu(self.conv2(x, data.edge_index))
            batch.append(self.fc(x.squeeze(0)))
        return torch.stack(batch, dim=0)

class CrossModalTransformer(nn.Module):
    def __init__(self, embed_dim=768):
        super().__init__()
        self.fc_phenotype = nn.Linear(256, embed_dim)
        self.cross_attn = nn.MultiheadAttention(embed_dim=embed_dim, num_heads=8)
        self.norm = nn.LayerNorm(embed_dim)

    def forward(self, smri, phenotype):
        pheno_proj = self.fc_phenotype(phenotype)
        combined = torch.stack([smri, pheno_proj], dim=0)
        out, _ = self.cross_attn(combined, combined, combined)
        return self.norm(out[0] + out[1])

class NeuralEnsemble(nn.Module):
    def __init__(self, input_dim=768, hidden_dim=128, num_classes=2, num_estimators=5):
        super().__init__()
        self.models = nn.ModuleList([
            nn.Sequential(
                nn.Linear(input_dim, hidden_dim),
                nn.ReLU(),
                nn.Dropout(0.3),
                nn.Linear(hidden_dim, num_classes))
            for _ in range(num_estimators)])

    def forward(self, x, training=True):
        preds = []
        for m in self.models:
            m.train(training)
            preds.append(m(x))
        outputs = torch.stack(preds)
        return outputs.mean(0), outputs.std(0).mean(-1)

class TransGraphASD(nn.Module):
    def __init__(self, image_size=128, patch_size=16, phenotype_features=10):
        super().__init__()
        self.mv_vit = MultiViewViT(image_size, patch_size)
        self.gnn = PhenotypeGNN(phenotype_features)
        self.cmt = CrossModalTransformer()
        self.ensemble = NeuralEnsemble()

    def forward(self, smri, pheno, training=True):
        logs = []

        with ResourceMonitor("MultiViewViT") as m:
            smri_feat = self.mv_vit(smri)
        logs.append(m.get_metrics())

        with ResourceMonitor("PhenotypeGNN") as m:
            pheno_feat = self.gnn(pheno)
        logs.append(m.get_metrics())

        with ResourceMonitor("CrossModalTransformer") as m:
            fused = self.cmt(smri_feat, pheno_feat)
        logs.append(m.get_metrics())

        with ResourceMonitor("NeuralEnsemble") as m:
            out, unc = self.ensemble(fused, training)
        logs.append(m.get_metrics())

        return out, unc, logs

# Contrastive loss

def contrastive_loss_fn(a, b):
    return -torch.mean(F.cosine_similarity(a, b, dim=-1))

# Training + inference

def print_resource_summary(resource_logs, phase="Training"):
    print(f"\n{phase} Resource Usage Summary\n" + "-" * 40)
    df = pd.DataFrame(resource_logs)
    for mod in df['module'].unique():
        sub = df[df['module'] == mod]
        print(f"{mod}: CPU Avg = {sub['cpu_usage_percent'].mean():.2f}%, RAM Avg = {sub['memory_usage_mb'].mean():.2f} MB")


def train_model(model, smri_data, pheno_data, labels, num_epochs=1, batch_size=4):
    tracker = EmissionsTracker(project_name="TransGraph-ASD Training", output_dir=".", output_file="emissions.csv")
    tracker.start()

    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    criterion = nn.CrossEntropyLoss()
    resource_logs = []

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        for i in range(0, len(smri_data), batch_size):
            batch_smri = smri_data[i:i+batch_size]
            batch_pheno = pheno_data[i:i+batch_size]
            batch_labels = labels[i:i+batch_size]

            optimizer.zero_grad()
            outputs, _, logs = model(batch_smri, batch_pheno)
            loss = criterion(outputs, batch_labels)

            smri_feat = model.mv_vit(batch_smri)
            pheno_feat = model.gnn(batch_pheno)
            pheno_proj = model.cmt.fc_phenotype(pheno_feat)
            c_loss = contrastive_loss_fn(smri_feat, pheno_proj)
            loss += 0.1 * c_loss
            loss.backward()
            optimizer.step()
            resource_logs.extend(logs)

        print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")
        print_resource_summary(resource_logs, phase=f"Epoch {epoch+1}")

    tracker.stop()
    return resource_logs


def predict_with_uncertainty(model, smri_data, pheno_data, mc_samples=3):
    model.eval()
    preds, uncs, logs = [], [], []
    with torch.no_grad():
        for _ in range(mc_samples):
            p, u, l = model(smri_data, pheno_data, training=True)
            preds.append(F.softmax(p, dim=-1))
            uncs.append(u)
            logs.extend(l)
    print_resource_summary(logs, phase="Inference")
    return torch.mean(torch.stack(preds), dim=0), torch.mean(torch.stack(uncs), dim=0), logs

# Main
if __name__ == "__main__":
    N, H, W, features = 32, 128, 128, 20
    smri = torch.randn(N, 3, 1, H, W)
    pheno = preprocess_phenotype_data(np.random.randn(N, features), [f"f{i}" for i in range(features)])
    labels = torch.randint(0, 2, (N,))

    model = TransGraphASD(image_size=H, patch_size=16, phenotype_features=features)
    train_model(model, smri, pheno, labels)
    preds, uncs, _ = predict_with_uncertainty(model, smri, pheno)
    print("\nPredictions:", preds)
    print("Uncertainties:", uncs)


c:\Users\chand\Desktop\Projects\vitgnn\vitgnn\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[codecarbon WARNING @ 09:05:45] Multiple instances of codecarbon are allowed to run at the same time.
[codecarbon INFO @ 09:05:45] [setup] RAM Tracking...
[codecarbon INFO @ 09:05:45] [setup] CPU Tracking...
[codecarbon WARNING @ 09:05:49] No CPU tracking mode found. Falling back on estimation based on TDP for CPU. 
 Windows OS detected: Please install Intel Power Gadget to measure CPU

[codecarbon INFO @ 09:05:49] CPU Model on constant consumption mode: 11th Gen Intel(R) Core(TM) i5-1135G7 @ 2.40GHz
[codecarbon WARNING @ 09:05:49] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 09:05:49] [setup] GPU Tracking...
[codecarbon INFO @ 09:05:49] No GPU found.
[codecarbon INFO @ 09:05:49] The b

[DEBUG] MultiViewViT: start_cpu=67.9, end_cpu=98.2, start_mem=704.02, end_mem=1020.79, duration=1.39s
[DEBUG] PhenotypeGNN: start_cpu=98.0, end_cpu=100.0, start_mem=1020.79, end_mem=1021.49, duration=0.16s
[DEBUG] CrossModalTransformer: start_cpu=100.0, end_cpu=100.0, start_mem=1021.49, end_mem=1021.76, duration=0.12s
[DEBUG] NeuralEnsemble: start_cpu=100.0, end_cpu=100.0, start_mem=1005.40, end_mem=985.32, duration=0.13s
[DEBUG] MultiViewViT: start_cpu=98.2, end_cpu=100.0, start_mem=1244.87, end_mem=1277.12, duration=1.28s


[codecarbon INFO @ 09:06:05] Energy consumed for RAM : 0.000042 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 09:06:05] Delta energy consumed for CPU with constant : 0.000470 kWh, power : 112.0 W
[codecarbon INFO @ 09:06:05] Energy consumed for All CPU : 0.000470 kWh
[codecarbon INFO @ 09:06:05] 0.000512 kWh of electricity used since the beginning.


[DEBUG] PhenotypeGNN: start_cpu=100.0, end_cpu=100.0, start_mem=1277.50, end_mem=1278.34, duration=0.15s
[DEBUG] CrossModalTransformer: start_cpu=100.0, end_cpu=100.0, start_mem=1279.55, end_mem=1279.70, duration=0.12s
[DEBUG] NeuralEnsemble: start_cpu=100.0, end_cpu=100.0, start_mem=1279.70, end_mem=1279.88, duration=0.12s
[DEBUG] MultiViewViT: start_cpu=100.0, end_cpu=95.9, start_mem=1368.69, end_mem=1375.09, duration=1.15s
[DEBUG] PhenotypeGNN: start_cpu=94.8, end_cpu=87.9, start_mem=1375.09, end_mem=1375.05, duration=0.12s
[DEBUG] CrossModalTransformer: start_cpu=94.1, end_cpu=91.7, start_mem=1375.07, end_mem=1375.21, duration=0.12s
[DEBUG] NeuralEnsemble: start_cpu=92.2, end_cpu=91.9, start_mem=1375.21, end_mem=1375.25, duration=0.12s


[codecarbon INFO @ 09:06:20] Energy consumed for RAM : 0.000083 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 09:06:20] Delta energy consumed for CPU with constant : 0.000466 kWh, power : 112.0 W
[codecarbon INFO @ 09:06:20] Energy consumed for All CPU : 0.000936 kWh
[codecarbon INFO @ 09:06:20] 0.001019 kWh of electricity used since the beginning.


...

[DEBUG] MultiViewViT: start_cpu=93.8, end_cpu=67.3, start_mem=1373.40, end_mem=1379.45, duration=0.92s
[DEBUG] PhenotypeGNN: start_cpu=88.3, end_cpu=77.4, start_mem=1379.45, end_mem=1379.46, duration=0.13s
[DEBUG] CrossModalTransformer: start_cpu=78.0, end_cpu=71.2, start_mem=1379.46, end_mem=1379.54, duration=0.12s
[DEBUG] NeuralEnsemble: start_cpu=84.2, end_cpu=80.8, start_mem=1379.54, end_mem=1379.61, duration=0.12s
[DEBUG] MultiViewViT: start_cpu=75.0, end_cpu=71.7, start_mem=1374.26, end_mem=1380.28, duration=0.86s
[DEBUG] PhenotypeGNN: start_cpu=59.6, end_cpu=70.7, start_mem=1380.28, end_mem=1380.38, duration=0.14s
[DEBUG] CrossModalTransformer: start_cpu=61.2, end_cpu=63.5, start_mem=1380.39, end_mem=1380.46, duration=0.12s
[DEBUG] NeuralEnsemble: start_cpu=77.2, end_cpu=84.0, start_mem=1380.46, end_mem=1380.49, duration=0.12s
[DEBUG] MultiViewViT: start_cpu=69.6, end_cpu=84.0, start_mem=1375.27, end_mem=1381.32, duration=0.88s
[DEBUG] PhenotypeGNN: start_cpu=79.4, end_cpu=90.6,

[codecarbon INFO @ 09:06:50] Energy consumed for RAM : 0.000166 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 09:06:50] Delta energy consumed for CPU with constant : 0.000467 kWh, power : 112.0 W
[codecarbon INFO @ 09:06:50] Energy consumed for All CPU : 0.001869 kWh
[codecarbon INFO @ 09:06:50] 0.002035 kWh of electricity used since the beginning.


[DEBUG] NeuralEnsemble: start_cpu=100.0, end_cpu=88.9, start_mem=1381.43, end_mem=1381.50, duration=0.12s


[codecarbon INFO @ 09:06:54] Energy consumed for RAM : 0.000179 kWh. RAM Power : 10.0 W
[codecarbon INFO @ 09:06:54] Delta energy consumed for CPU with constant : 0.000147 kWh, power : 112.0 W
[codecarbon INFO @ 09:06:54] Energy consumed for All CPU : 0.002016 kWh
[codecarbon INFO @ 09:06:54] 0.002195 kWh of electricity used since the beginning.


Epoch 1, Loss: 0.6873

Epoch 1 Resource Usage Summary
----------------------------------------
MultiViewViT: CPU Avg = 87.23%, RAM Avg = 48.25 MB
PhenotypeGNN: CPU Avg = 85.54%, RAM Avg = 0.22 MB
CrossModalTransformer: CPU Avg = 84.58%, RAM Avg = 0.12 MB
NeuralEnsemble: CPU Avg = 90.44%, RAM Avg = 0.06 MB
[DEBUG] MultiViewViT: start_cpu=74.5, end_cpu=93.9, start_mem=1375.38, end_mem=1045.81, duration=4.79s
[DEBUG] PhenotypeGNN: start_cpu=95.0, end_cpu=89.8, start_mem=1045.84, end_mem=1045.85, duration=0.23s
[DEBUG] CrossModalTransformer: start_cpu=88.1, end_cpu=89.7, start_mem=1045.85, end_mem=1045.62, duration=0.12s
[DEBUG] NeuralEnsemble: start_cpu=96.2, end_cpu=100.0, start_mem=1045.62, end_mem=1045.71, duration=0.12s
[DEBUG] MultiViewViT: start_cpu=100.0, end_cpu=77.2, start_mem=1045.71, end_mem=1045.66, duration=5.73s
[DEBUG] PhenotypeGNN: start_cpu=74.5, end_cpu=98.1, start_mem=1045.66, end_mem=1045.67, duration=0.22s
[DEBUG] CrossModalTransformer: start_cpu=77.2, end_cpu=86.7, s